In [ ]:
from openai import OpenAI
import pandas as pd
import json
import os

client = OpenAI(
  api_key="REDACTED"
)


In [ ]:
# 질문에 대한 모델의 답변 생성 함수
def generate_answer(model, expand_context, question, gold_answer, answer):

    prompt_improved = f"""
      LLM 응답 품질 평가 전문가입니다. 답변을 전문가 검수 정답과 비교하여 평가합니다.

      **평가 대상:**
      - 질문: {question}
      - 평가할 답변: {answer}
      - 정답 (전문가 검수): {gold_answer}

      **평가 방식:**
      1. 정답이 사용한 문서 정보를 파악
      2. 답변이 동일한 정보와 논리를 사용했는지 비교
      3. 결론이 정답과 다르면 감점

      **중요: 아래 참조 문서에서 정답이 어떤 정보를 사용했는지 반드시 확인하세요.**

      **참조 문서:**
      {expand_context}

      ---

      위 문서를 기반으로 다음 평가 기준에 따라 점수를 매기세요.

      **평가 기준 (각 1-10점):**

      1. **정합성**: 정답이 사용한 관련 정보를 답변도 사용했는가?
        - 1-3: 정답과 다른 무관한 정보 다수 사용, 핵심 문맥 왜곡
        - 4-6: 일부 정답과 같은 정보 사용하나 무관한 내용도 포함
        - 7-8: 대부분 정답과 동일한 정보 사용, 소수 부차적 정보 혼입
        - 9-10: 정답과 동일한 정보만 선별적 활용, 불필요한 문맥 배제

      2. **일관성**: 정답처럼 주제 초점이 흔들리지 않는가?
        - 1-3: 주제가 빈번히 전환됨, 정답과 다른 방향
        - 4-6: 전반적 일관성 있으나 비관련 내용 간헐적 혼입
        - 7-8: 정답과 유사하게 톤과 논점 유지, 약간의 주변 정보
        - 9-10: 정답처럼 처음부터 끝까지 주제 집중, 논리적 일관성

      3. **정확성**: 정답의 사실과 일치하는가?
        - 1-3: 정답과 다른 사실 제시, 무관한 정보를 사실로 제시
        - 4-6: 주요 사실은 정답과 맞으나 일부 세부 오류
        - 7-8: 정답과 전반적으로 일치, 약간의 모호함
        - 9-10: 정답의 사실과 완전히 일치

      4. **완전성**: 정답이 다룬 핵심 쟁점을 빠짐없이 다루었는가?
        - 1-3: 정답의 핵심 요소 대부분 누락
        - 4-6: 정답의 주요 쟁점은 언급했으나 하위 맥락 부족
        - 7-8: 정답의 대부분 측면을 다루나 부차적 요소 약함
        - 9-10: 정답처럼 모든 측면을 빠짐없이 포괄적으로 다룸

      5. **추론성**: 정답과 유사한 논리적 추론 과정을 보이는가?
        - 1-3: 정답과 다른 추론, 과정 없거나 비논리적
        - 4-6: 일부 정답과 유사하나 논리 비약 발생
        - 7-8: 정답과 대부분 유사한 자연스러운 추론
        - 9-10: 정답과 동일한 단계별 명확한 논리 흐름

      6. **전체 품질**: 정답과의 전반적 일치도
        - 1-3: 정답과 큰 차이, 전반적 품질 낮음
        - 4-6: 일정 수준 일치하나 논리 흐름 불완전
        - 7-8: 정답과 유사하게 잘 구성, 소수 불일치
        - 9-10: 정답과 높은 일치도, 완성도 높음

      **출력 형식 (JSON):**
      {{
        "coherence": 점수,
        "consistency": 점수,
        "accuracy": 점수,
        "completeness": 점수,
        "reasoning": 점수,
        "overall_quality": 점수,
        "explanation": "정답과의 주요 차이점과 일치 정도 요약 (2-3문장)"
      }}
      """

    completion = client.chat.completions.create(
      model=model,
      store=True,
      messages=[{"role": "system", "content": "You are a financial domain expert. Your task is to evaluate the clarity and difficulty "
                      "of the given LLM response based on your expertise in finance."},
                    {"role": "user", "content": prompt_improved}],
    )

    return completion.choices[0].message.content

In [ ]:
from tqdm import tqdm

# CSV 파일 처리
def process_csv(input_csv_path, output_csv_path):
    # 입력 CSV 파일 읽기
    data = pd.read_csv(input_csv_path)

    # 결과 저장용 리스트
    results = []
    num = 1
    error_ids = []

    headers = data.columns.tolist()
    for index, row in tqdm(data.iterrows()):
        try:
          _id = row['id']

          question = row['question']
          gold_answer = ['gold_answer']

          shuffled_balance_text = row['shuffled_balance_text']
          shuffled_cluster_front_text = row['shuffled_cluster_front_text']
          shuffled_cluster_middle_text = row['shuffled_cluster_middle_text']
          shuffled_cluster_end_text = row['shuffled_cluster_end_text']
          shuffled_random_text = row['shuffled_random_text']

          answer_balance = row['gpt-oss:120b_answer_balance']
          answer_front = row['gpt-oss:120b_answer_front']
          answer_middle = row['gpt-oss:120b_answer_middle']
          answer_end = row['gpt-oss:120b_answer_end']
          answer_random = row['gpt-oss:120b_answer_random']

          # gpt-5 모델 평가답변 생성
          answer_balance_eval = generate_answer('gpt-5', shuffled_balance_text, question, gold_answer, answer_balance)

          answer_front_eval = generate_answer('gpt-5', shuffled_cluster_front_text, question, gold_answer, answer_front)

          answer_middle_eval = generate_answer('gpt-5', shuffled_cluster_middle_text, question, gold_answer, answer_middle)

          answer_end_eval = generate_answer('gpt-5', shuffled_cluster_end_text, question, gold_answer, answer_end)

          answer_random_eval = generate_answer('gpt-5', shuffled_random_text, question, gold_answer, answer_random)



          # 결과 저장 -> 각 모델별로
          results.append({
              "id": _id,
              "answer_balance_eval": answer_balance_eval,
              "answer_front_eval": answer_front_eval,
              "answer_middle_eval": answer_middle_eval,
              "answer_end_eval": answer_end_eval,
              "answer_random_eval": answer_random_eval,
          })

          num += 1

        except Exception as e:
          print("error : ", e)
          error_ids.append(index)
          num += 1
          continue

    print(f"{num}개 데이터 평가 완료")

    # 결과를 DataFrame으로 변환 후 CSV 파일로 저장
    results_df = pd.DataFrame(results)
    results_df.to_csv(output_csv_path, index=False, encoding='utf-8')

In [ ]:

# 실행 예시
if __name__ == "__main__":
    # 예제 데이터 경로
    file_path = "./drive/MyDrive/Colab Notebooks/독자AI/추론/12번_fn"
    file_list = os.listdir(file_path)


    for file_name in file_list:
      print(file_name)
      if "gpt-oss" not in file_name:
        continue
      print(f"{file_name} Start")
      input_csv_path = f"{file_path}/{file_name}"   # 입력 CSV 파일 경로
      output_csv_path = f"./drive/MyDrive/Colab Notebooks/독자AI/추론/12번_fn_답변평가/{file_name}_gpt-5_eval_v2.csv"  # 출력 CSV 파일 경로 (결과 저장용)

      # CSV 처리 실행
      process_csv(input_csv_path, output_csv_path)

      print(f"{file_name} Done")
      print()



(FN)12_70건_전처리 및 셔플 완료.csv_qwen3_30b-a3b-instruct-2507-fp16-reasoning.csv
(FN)12_70건_전처리 및 셔플 완료.csv_deepseek-r1_70b_reasoning.csv
(FN)12_70건_전처리 및 셔플 완료.csv_gpt-oss-120b-reasoning.csv
(FN)12_70건_전처리 및 셔플 완료.csv_gpt-oss-120b-reasoning.csv Start


70it [3:04:12, 157.89s/it]

71개 데이터 평가 완료
(FN)12_70건_전처리 및 셔플 완료.csv_gpt-oss-120b-reasoning.csv Done

